In [6]:
import os
import json
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

# .env 파일 로드
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. StructuredOutputParser를 위한 ResponseSchema 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지 이름"),
    ResponseSchema(name="duration", description="여행 기간"),
    ResponseSchema(name="budget", description="여행에 사용된 총 예산"),
    ResponseSchema(name="rating", description="여행에 대한 만족도 (1점-5점 사이의 숫자)"), # Added "숫자"
    ResponseSchema(name="activities", description="여행에서 한 주요 활동들을 포함하는 문자열 리스트") # Added "문자열 리스트"
]

# 2. StructuredOutputParser 초기화
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 3. 모델 초기화
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  
    model="meta-llama/llama-4-scout-17b-16e-instruct",  
    temperature=0.7
)

# 4. 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="""주어진 여행 후기에서 다음 정보를 추출하세요: 여행지, 기간, 예산, 추천도, 주요 활동.
{format_instructions}
여행 후기: "{review}" """,
    input_variables=["review"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

# 5. LLM Chain 구성
chain = prompt | llm | output_parser

# 6. 테스트 입력
input_review = "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

# 7. 실행 및 결과 출력
try:
    parsed_output = chain.invoke({"review": input_review})
    print("--- 여행 후기 분석 결과 ---")
    print(json.dumps(parsed_output, indent=4, ensure_ascii=False))

except Exception as e:
    print(f"정보 추출 실패: {e}")

--- 여행 후기 분석 결과 ---
{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4",
    "activities": [
        "해운대 바다구경",
        "자갈치시장에서 회 먹기",
        "감천문화마을 구경"
    ]
}
